In [38]:
from load_data import *
from bs4 import BeautifulSoup
import webbrowser
import requests
import pandas as pd
import urllib.request as urli
import os 



# Load listings

In [39]:
url_listing = "http://data.insideairbnb.com/ireland/leinster/dublin/2021-11-07/data/listings.csv.gz"
listings = pd.read_csv(url_listing)
prefix = "https://airbnb.de"
suffix = "/photos"
urls = listings["listing_url"]


In [41]:
for id, url in enumerate(urls):
    link = requests.get(url+suffix)
    html_content = BeautifulSoup(link.content, "html.parser")
    gallery_items = html_content.findAll("img", {"class": "_6tbg2q"})

    for i, item in enumerate(gallery_items):
        img_link = item.get("src")
        path_name = "images/"+str(id)+"_"+str(i)+".png"
        if os.path.exists(path_name):
            continue
        else:
            urli.urlretrieve(img_link, path_name)
        

KeyboardInterrupt: 